# Machine learning - Predict MatchDay

## 0. Imports

In [3]:
import pandas as pd
import math
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import os

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## 1. Filter & Format Barcelona Dataset

### 1.1 Read in csv-files 

In [4]:
#Read in CSV's in folder
totalFileList = os.listdir(
        "/home/woutd/Documents/School/Industriele Ingenieurswetenschappen/Ma1/Sem1/Machine Learning/Opdracht/Workspace/2-Twitter/")
listFileNames = []
for fileName in totalFileList:
    if ('Barcelona' in fileName):
        listFileNames.append(fileName)       
datasets = []
for fileName in  listFileNames:
    datasets.append(pd.read_csv("/home/woutd/Documents/School/Industriele Ingenieurswetenschappen/Ma1/Sem1/Machine Learning/Opdracht/Workspace/2-Twitter/" + str(fileName)))


['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count']


### 1.2 Merge datasets into 1

In [ ]:
merged_dataset = pd.DataFrame()
for dataset in datasets:
    merged_dataset = pd.concat([merged_dataset, dataset])
merged_dataset.drop_duplicates()

print(list(merged_dataset.columns))

### 1.3 Format dataset [date, count]

In [5]:
def extractDate(value):
    date = pd.to_datetime(value)
    return date.date()

def incr(value):
    return value + 1

In [6]:
merged_dataset['dates'] = merged_dataset['timestamp'].apply(extractDate)

dataset = pd.DataFrame(columns=['Date', 'Count'])
for date in merged_dataset['dates']:
    if(date in dataset['Date']):
        
        print('Great Succes')
    else:
        dataset = dataset.append({'Date': date, 'Count': 0}, ignore_index=True)
print(dataset)

NameError: name 'originalDataset' is not defined